### NPLM概述

NPLM是一种将词汇映射到连续向量空间的方法，其核心思想是利用神经网络结构学习词汇的概率分布。NPLM通过利用前N-1个词来预测第N个词，这与N-Gram相似，但结构远远要比N-Gram要复杂


### NPLM结构

NPLM的结构包括三个部分：输入层、隐藏层和输出层。输入层将词汇映射到连续的向量空间(Embedding),隐藏层通过非线性激活函数学习词与词之间的复杂关系，输出层通过softmax函数产生下一个单词出现的概率，具体结构如图:

<img src="./images/NPLM.png" alt="img" style="zoom: 30%;" />

### NPLM的实现

> 1. 构建实验语料

In [140]:
sentences = ["我 喜欢 玩具", "我 爱 爸爸", "我 讨厌 挨打"] 
# 创建词汇表
word_list = list(set(" ".join(sentences).split()))
# index -> word
index_to_word = { index: word for index,word in enumerate(word_list)}
word_to_index = { word: index for index,word in enumerate(word_list)}
print("词汇表长度",len(word_list))
print(word_to_index)

词汇表长度 7
{'爸爸': 0, '我': 1, '挨打': 2, '喜欢': 3, '爱': 4, '讨厌': 5, '玩具': 6}


> 2. 生成NPLM数据

In [141]:
# 从语料库中生成批处理数据，一批一批的喂入网络进行训练
import torch
import random
batch_size = 2
def make_batch(batch_size):
    input_batch = []
    target_batch = []
    sample_sentences = random.sample(sentences,batch_size) # 通过random随机选择数据
    # print(sample_sentences)
    for sentence in sample_sentences:
        # 通过空格分隔词汇
        split_sentences = sentence.split()
        input = [word_to_index[ele] for ele in split_sentences[:-1]] # 将前n-1个词汇的索引作为输入
        target = word_to_index[split_sentences[-1]] # 目标词汇为最后一个词
        input_batch.append(input)
        target_batch.append(target)
    input_batch = torch.LongTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)
    return input_batch, target_batch
input_batch, targe_batch = make_batch(batch_size)
print("打印训练数据", input_batch)

打印训练数据 tensor([[1, 4],
        [1, 5]])


> 3. 构建NPLM模型

In [142]:
import torch.nn as nn

class NPLM(nn.Module):
    def __init__(self, voc_size, embedding_size, n_hidden, n_step):
        super(NPLM, self).__init__()
        self.C = nn.Embedding(voc_size, embedding_size) # 定义一个词嵌入层
        # 第一个线性层的输入是将前N-1个词汇的嵌入表示平铺:n_step * embedding_size，输出为隐藏层大小
        self.l1 = nn.Linear(n_step * embedding_size, n_hidden)
        # 第二个线性层的输出为voc_size
        self.l2 = nn.Linear(n_hidden, voc_size)
    def forward(self,X):
        # 输入张量X的形状为[batch_size, n_step]
        X = self.C(X) # [batch_size, n_step, embedding_size]
        X = X.view(-1 ,n_step * embedding_size) # [batch_size, n_step * embedding_size]
        hidden = torch.tanh(self.l1(X)) # [batch_size, n_hidden]
        output = self.l2(hidden) # [batch_size, voc_size]
        return output


In [143]:
voc_size = len(word_list)
n_step = 2 # 时间步数，表示每个输入序列的长度，也就是上下文长度 
n_hidden = 2 # 隐藏层大小
embedding_size = 2 # 词嵌入大小
model = NPLM(voc_size, embedding_size, n_hidden, n_step) # 创建神经概率语言模型实例
print(' NPLM 模型结构：', model) # 打印模型的结构

 NPLM 模型结构： NPLM(
  (C): Embedding(7, 2)
  (l1): Linear(in_features=4, out_features=2, bias=True)
  (l2): Linear(in_features=2, out_features=7, bias=True)
)


> 4. 模型训练

In [147]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
# 训练模型
for epoch in range(5000):
    optimizer.zero_grad()
    input, target = make_batch(2)
    output = model(input)
    loss = criterion(output, target)
    if (epoch + 1) % 1000 == 0: # 每 1000 次迭代，打印损失值
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.000006
Epoch: 2000 cost = 0.000002
Epoch: 3000 cost = 0.000001
Epoch: 4000 cost = 0.000000
Epoch: 5000 cost = 0.000000


In [145]:
import torch

# 创建一个维度为 (1, 3, 1, 4) 的张量
x = torch.randn(1, 3, 1, 4)
print("原始张量维度：", x.shape)  # 输出：torch.Size([1, 3, 1, 4])
print(x)

# 使用 squeeze() 方法去除大小为 1 的维度
x_squeezed = x.squeeze()
print("去除维度后的张量维度：", x_squeezed.shape)  # 输出：torch.Size([3, 4])
print(x_squeezed)


原始张量维度： torch.Size([1, 3, 1, 4])
tensor([[[[-1.5833, -0.4457, -0.8972,  0.8893]],

         [[ 0.3489, -1.5189, -1.0801,  1.3319]],

         [[ 0.1954, -2.3678,  0.3898, -0.1858]]]])
去除维度后的张量维度： torch.Size([3, 4])
tensor([[-1.5833, -0.4457, -0.8972,  0.8893],
        [ 0.3489, -1.5189, -1.0801,  1.3319],
        [ 0.1954, -2.3678,  0.3898, -0.1858]])


> 5. 预测

In [158]:
# 进行预测
input_strs = [['我', '讨厌'], ['我', '喜欢']]  # 需要预测的输入序列
# 将输入序列转换为对应的索引
input_indices = [[word_to_index[word] for word in seq] for seq in input_strs]
# 将输入序列的索引转换为张量
print(input_indices)
input_batch = torch.LongTensor(input_indices) 
# 对输入序列进行预测，取输出中概率最大的类别
# .data从模型输出中提取实际的张量数据, .max(1)获得沿词汇维度张量数据的最大值并返回其最大值和对应的索引, 因此[1]表示对的索引
predict = model(input_batch).data.max(1)[1]
print(predict)
# 将预测结果的索引转换为对应的词
predict_strs = [index_to_word[n.item()] for n in predict.squeeze()]  
for input_seq, pred in zip(input_strs, predict_strs):
   print(input_seq, '->', pred)  # 打印输入序列和预测结果

[[1, 5], [1, 3]]
tensor([2, 6])
['我', '讨厌'] -> 挨打
['我', '喜欢'] -> 玩具


### NPLM的优缺点
优点

- 可以自动学习复杂的特征表示，减少了手工特征工程的需求
- 对大量数据进行高效的处理，学习词与词之间的语义和语法关系
- 具有强大的拟合能力
  
缺点

- 模型结构简单，仍然属于浅层的神经元网络。对于复杂的语言模式和长距离依赖关系，就无法捕捉到足够大信息
- 窗口大小固定，也就是线性层的大小为n_step * embedding_size固定了输入序列的大小，限制了模型处理不同长度上下文的能力
- 词汇表固定，无法处理训练集中未出现的词汇，泛化能力较弱
- 缺乏位置信息，没有考虑输入序列中单词的顺序，使模型无法捕捉词与词之间的顺序关系网